In [109]:
import theano
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score, KFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

seed = 7
np.random.seed(seed)

In [47]:
df = pd.read_csv('https://raw.githubusercontent.com/oneilltp18/GABBERT/master/wide_receivers/refined_training_set.csv', index_col = 'name')


In [48]:
df.categories.value_counts()

below average      220
league_average      51
quality starter     37
all_pro             23
Name: categories, dtype: int64

In [49]:
X = np.asmatrix(df.drop('categories', axis = 1))
y = np.asmatrix(pd.get_dummies(df.categories.map({'below average':0, 'league_average':1, 'quality starter':2, 'all_pro':3})))

encoder = LabelEncoder()
encoder.fit(df.categories)
enc_y = encoder.transform(df.categories) 
dummy_y = np_utils.to_categorical(enc_y)

print X.shape
print y.shape
y

(331, 30)
(331, 4)


matrix([[ 0.,  0.,  1.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        ..., 
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.]])

# Create ANN Model

In [213]:
def baseline_model():
    ## creating model
    model = Sequential()
    model.add(Dense(100, input_dim = 30, init = 'uniform', activation = 'relu'))
#     model.add(Dropout(.2))
#     model.add(Dense(100, init = 'uniform', activation = 'softmax'))
#    model.add(Dropout(.3))
#    model.add(Dense(60, init = 'uniform', activation = 'relu'))
#    model.add(Dense(40, init = 'uniform', activation = 'relu'))
#    model.add(Dense(20, init = 'uniform', activation = 'relu'))
    model.add(Dense(4, init = 'uniform', activation = 'sigmoid'))
    
    ## compile model
    model.compile(loss='categorical_crossentropy', optimizer='adadelta',
              metrics=['accuracy'])
    return model


In [214]:
estimator = KerasClassifier(build_fn = baseline_model, nb_epoch = 1000,
                            batch_size = 5, verbose = 0)

In [215]:
kfold = KFold(n = len(X), n_folds = 4, shuffle = True, random_state = seed)

In [216]:
results = cross_val_score(estimator, X, y, cv = kfold)
print ("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 63.74% (2.06%)


In [217]:
y_pred = cross_val_predict(estimator, X, y, cv = kfold)

print classification_report(enc_y, y_pred)

             precision    recall  f1-score   support

          0       0.06      0.74      0.11        23
          1       0.37      0.05      0.09       220
          2       0.25      0.06      0.10        51
          3       0.55      0.16      0.25        37

avg / total       0.35      0.11      0.11       331

